In [77]:
!kubectl apply -f .

statefulset.apps/mongodb-ss created
secret/mongodb-colordb-creds created
secret/mongodb-root-creds created
configmap/mongodb-init-colordb created
service/mongodb-svc created


In [78]:
!kubectl get svc

NAME          TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
kubernetes    ClusterIP   10.96.0.1    <none>        443/TCP     3d1h
mongodb-svc   ClusterIP   None         <none>        27017/TCP   40s


In [79]:
!kubectl get configmap,secret

NAME                             DATA   AGE
configmap/kube-root-ca.crt       1      3d1h
configmap/mongodb-init-colordb   1      42s

NAME                           TYPE     DATA   AGE
secret/mongodb-colordb-creds   Opaque   2      42s
secret/mongodb-root-creds      Opaque   2      42s


In [80]:
!kubectl get pods

NAME           READY   STATUS    RESTARTS   AGE
mongodb-ss-0   1/1     Running   0          47s


In [81]:
!kubectl describe statefulset

Name:               mongodb-ss
Namespace:          default
CreationTimestamp:  Sun, 31 Aug 2025 21:25:33 -0400
Selector:           app=mongodb
Labels:             <none>
Annotations:        <none>
Replicas:           1 desired | 1 total
Update Strategy:    RollingUpdate
  Partition:        0
Pods Status:        1 Running / 0 Waiting / 0 Succeeded / 0 Failed
Pod Template:
  Labels:  app=mongodb
  Containers:
   mongo:
    Image:      mongo:8.0.0
    Port:       27017/TCP
    Host Port:  0/TCP
    Environment:
      MONGO_INITDB_ROOT_USERNAME:  <set to the key 'username' in secret 'mongodb-root-creds'>  Optional: false
      MONGO_INITDB_ROOT_PASSWORD:  <set to the key 'password' in secret 'mongodb-root-creds'>  Optional: false
      DB_NAME:                     colordb
      DB_USER:                     <set to the key 'username' in secret 'mongodb-colordb-creds'>  Optional: false
      DB_PASSWORD:                 <set to the key 'password' in secret 'mongodb-colordb-creds'>  Optional:

In [15]:
!kubectl describe pod mongodb-ss-0

Name:             mongodb-ss-0
Namespace:        default
Priority:         0
Service Account:  default
Node:             docker-desktop/192.168.65.3
Start Time:       Sun, 31 Aug 2025 20:49:43 -0400
Labels:           app=mongodb
                  apps.kubernetes.io/pod-index=0
                  controller-revision-hash=mongodb-ss-7cf67cff74
                  statefulset.kubernetes.io/pod-name=mongodb-ss-0
Annotations:      <none>
Status:           Running
IP:               10.1.1.181
IPs:
  IP:           10.1.1.181
Controlled By:  StatefulSet/mongodb-ss
Containers:
  mongo:
    Container ID:   docker://0f959f4d89981a90c061526d9cd4da0b78bd446bd12ba69dbe7bc5f0de52aac5
    Image:          mongo:8.0.0
    Image ID:       docker-pullable://mongo@sha256:0c17421487ae4a5cae88e7652a26ef72a8162adbc599a0730805526f5ff5ee83
    Port:           27017/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Sun, 31 Aug 2025 20:49:59 -0400
    Last State:     Terminated
      Reas

In [82]:
!kubectl logs mongodb-ss-0

about to fork child process, waiting until server is ready for connections.
forked process: 29

{"t":{"$date":"2025-09-01T01:25:35.921+00:00"},"s":"I",  "c":"CONTROL",  "id":20698,   "ctx":"main","msg":"***** SERVER RESTARTED *****"}
{"t":{"$date":"2025-09-01T01:25:35.921+00:00"},"s":"I",  "c":"CONTROL",  "id":23285,   "ctx":"main","msg":"Automatically disabling TLS 1.0, to force-enable TLS 1.0 specify --sslDisabledProtocols 'none'"}
{"t":{"$date":"2025-09-01T01:25:35.923+00:00"},"s":"I",  "c":"CONTROL",  "id":5945603, "ctx":"main","msg":"Multi threading initialized"}
{"t":{"$date":"2025-09-01T01:25:35.923+00:00"},"s":"I",  "c":"NETWORK",  "id":4648601, "ctx":"main","msg":"Implicit TCP FastOpen unavailable. If TCP FastOpen is required, set at least one of the related parameters","attr":{"relatedParameters":["tcpFastOpenServer","tcpFastOpenClient","tcpFastOpenQueueSize"]}}
{"t":{"$date":"2025-09-01T01:25:35.924+00:00"},"s":"I",  "c":"NETWORK",  "id":4915701, "ctx":"main","msg":"Initiali

In [84]:
!kubectl exec mongodb-ss-0 -- mongosh -u root_user -p root_password --authenticationDatabase admin --eval "db.stats()"


{
  db: 'test',
  collections: Long('0'),
  views: Long('0'),
  objects: Long('0'),
  avgObjSize: 0,
  dataSize: 0,
  storageSize: 0,
  indexes: Long('0'),
  indexSize: 0,
  totalSize: 0,
  scaleFactor: Long('1'),
  fsUsedSize: 0,
  fsTotalSize: 0,
  ok: 1
}


In [105]:
!kubectl exec mongodb-ss-0 -- mongosh -u colordb_user -p colordb_password --authenticationDatabase colordb --eval "db.getSiblingDB(\"colordb\").testCollection.insertOne({hello: \"world\"})"


{
  acknowledged: true,
  insertedId: ObjectId('68b4f9c5251d0f17fb964033')
}


In [106]:
!kubectl exec mongodb-ss-0 -- mongosh -u colordb_user -p colordb_password --authenticationDatabase colordb --eval "db.getSiblingDB(\"colordb\").testCollection.find()"


[
  { _id: ObjectId('68b4f7dbe81beb1ade964033'), hello: 'world' },
  { _id: ObjectId('68b4f9c5251d0f17fb964033'), hello: 'world' }
]


In [93]:
!kubectl get pods

NAME           READY   STATUS    RESTARTS   AGE
mongodb-ss-0   1/1     Running   0          10m


In [94]:
!kubectl scale statefulset mongodb-ss --replicas=5

statefulset.apps/mongodb-ss scaled


In [96]:
!kubectl get pods

NAME           READY   STATUS              RESTARTS   AGE
mongodb-ss-0   1/1     Running             0          10m
mongodb-ss-1   1/1     Running             0          10s
mongodb-ss-2   1/1     Running             0          6s
mongodb-ss-3   0/1     ContainerCreating   0          3s


In [107]:
!kubectl exec mongodb-ss-0 -- mongosh -u colordb_user -p colordb_password --authenticationDatabase colordb --eval "db.getSiblingDB(\"colordb\").testCollection.find()"


[
  { _id: ObjectId('68b4f7dbe81beb1ade964033'), hello: 'world' },
  { _id: ObjectId('68b4f9c5251d0f17fb964033'), hello: 'world' }
]


In [108]:
!kubectl exec mongodb-ss-1 -- mongosh -u colordb_user -p colordb_password --authenticationDatabase colordb --eval "db.getSiblingDB(\"colordb\").testCollection.find()"


In [109]:
!kubectl get pv,pvc

NAME                                                        CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                               STORAGECLASS   VOLUMEATTRIBUTESCLASS   REASON   AGE
persistentvolume/pvc-063ffbc4-e98c-4608-9897-d810aa369c68   5Gi        RWO            Delete           Bound    default/mongodb-data-mongodb-ss-0   hostpath       <unset>                          17m
persistentvolume/pvc-0c7dd438-05a6-4ea1-bbf1-9a522b241cea   5Gi        RWO            Delete           Bound    default/mongodb-data-mongodb-ss-4   hostpath       <unset>                          6m21s
persistentvolume/pvc-d3ce881d-bd19-46f0-8c9b-4608e56b34cf   5Gi        RWO            Delete           Bound    default/mongodb-data-mongodb-ss-3   hostpath       <unset>                          6m24s
persistentvolume/pvc-e770456d-dc05-4910-b31c-6bf8aa1a0a6d   5Gi        RWO            Delete           Bound    default/mongodb-data-mongodb-ss-1   hostpath       <unset>                          

In [110]:
!kubectl scale statefulset mongodb-ss --replicas=1

statefulset.apps/mongodb-ss scaled


In [111]:
!kubectl get pods

NAME           READY   STATUS    RESTARTS   AGE
mongodb-ss-0   1/1     Running   0          18m


In [112]:
!kubectl get pvc

NAME                        STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   VOLUMEATTRIBUTESCLASS   AGE
mongodb-data-mongodb-ss-0   Bound    pvc-063ffbc4-e98c-4608-9897-d810aa369c68   5Gi        RWO            hostpath       <unset>                 19m
mongodb-data-mongodb-ss-1   Bound    pvc-e770456d-dc05-4910-b31c-6bf8aa1a0a6d   5Gi        RWO            hostpath       <unset>                 8m45s
mongodb-data-mongodb-ss-2   Bound    pvc-fc89b0e2-ecc0-4d3c-b808-c0a26548a8c1   5Gi        RWO            hostpath       <unset>                 8m41s
mongodb-data-mongodb-ss-3   Bound    pvc-d3ce881d-bd19-46f0-8c9b-4608e56b34cf   5Gi        RWO            hostpath       <unset>                 8m38s
mongodb-data-mongodb-ss-4   Bound    pvc-0c7dd438-05a6-4ea1-bbf1-9a522b241cea   5Gi        RWO            hostpath       <unset>                 8m35s


In [113]:
!kubectl delete pvc mongodb-data-mongodb-ss-1 mongodb-data-mongodb-ss-2 mongodb-data-mongodb-ss-3 mongodb-data-mongodb-ss-4

persistentvolumeclaim "mongodb-data-mongodb-ss-1" deleted
persistentvolumeclaim "mongodb-data-mongodb-ss-2" deleted
persistentvolumeclaim "mongodb-data-mongodb-ss-3" deleted
persistentvolumeclaim "mongodb-data-mongodb-ss-4" deleted


In [114]:
!kubectl get pv,pvc

NAME                                                        CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                               STORAGECLASS   VOLUMEATTRIBUTESCLASS   REASON   AGE
persistentvolume/pvc-063ffbc4-e98c-4608-9897-d810aa369c68   5Gi        RWO            Delete           Bound    default/mongodb-data-mongodb-ss-0   hostpath       <unset>                          20m

NAME                                              STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   VOLUMEATTRIBUTESCLASS   AGE
persistentvolumeclaim/mongodb-data-mongodb-ss-0   Bound    pvc-063ffbc4-e98c-4608-9897-d810aa369c68   5Gi        RWO            hostpath       <unset>                 20m


In [9]:
!kubectl apply -f .

deployment.apps/color-api unchanged
service/color-svc configured
statefulset.apps/mongodb-ss configured
secret/mongodb-colordb-creds unchanged
secret/mongodb-root-creds unchanged
configmap/mongodb-init-colordb unchanged
service/mongodb-svc unchanged


In [10]:
!kubectl get pods -l app=color-api

NAME                         READY   STATUS    RESTARTS   AGE
color-api-7bf587b48f-wxwqm   1/1     Running   0          3m24s


In [4]:
!kubectl logs color-api-7bf587b48f-wxwqm


> color-api@1.0.0 start
> node src/index.js

Fail liveness: false
Fail readiness: false
Delayed startup: false
Connected to MongoDB
Color-API listening on port 80


In [11]:
!kubectl get svc

NAME          TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
color-svc     NodePort    10.111.72.33   <none>        80:30080/TCP   3m26s
kubernetes    ClusterIP   10.96.0.1      <none>        443/TCP        3d3h
mongodb-svc   ClusterIP   None           <none>        27017/TCP      133m


In [12]:
!kubectl describe svc color-sv

Name:                     color-svc
Namespace:                default
Labels:                   <none>
Annotations:              <none>
Selector:                 app=color-api
Type:                     NodePort
IP Family Policy:         SingleStack
IP Families:              IPv4
IP:                       10.111.72.33
IPs:                      10.111.72.33
Port:                     <unset>  80/TCP
TargetPort:               80/TCP
NodePort:                 <unset>  30080/TCP
Endpoints:                10.1.1.199:80
Session Affinity:         None
External Traffic Policy:  Cluster
Internal Traffic Policy:  Cluster
Events:                   <none>



In [13]:
!curl localhost:30080

<h1 style="color:blue;">Hello from Color-API!</h1>
            <h2>Hostname: color-api-7bf587b48f-wxwqm</h2>


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   108  100   108    0     0  19512      0 --:--:-- --:--:-- --:--:-- 21600


In [14]:
!curl localhost:30080/api

COLOR : blue, HOSTNAME : color-api-7bf587b48f-wxwqm


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    51  100    51    0     0   9908      0 --:--:-- --:--:-- --:--:-- 10200


In [15]:
!curl localhost:30080/api/color

{"data":[]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    11  100    11    0     0   1838      0 --:--:-- --:--:-- --:--:--  2200


In [26]:
!curl -X POST http://localhost:30080/api/color/primary -H "Content-Type: application/json" -d "{\"value\":\"purple\"}"

Created


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    25  100     7  100    18    352    906 --:--:-- --:--:-- --:--:--  1315


In [27]:
!curl localhost:30080/api/color

{"data":[{"_id":"68b5186ff8ee3e164f8be23d","key":"primary","value":"purple","__v":0}]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    86  100    86    0     0   2658      0 --:--:-- --:--:-- --:--:--  2687


In [28]:
!curl -X POST http://localhost:30080/api/color/secondary -H "Content-Type: application/json" -d "{\"value\":\"orange\"}"

Created


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    25  100     7  100    18    329    847 --:--:-- --:--:-- --:--:--  1190


In [30]:
!curl localhost:30080/api/color

{"data":[{"_id":"68b5186ff8ee3e164f8be23d","key":"primary","value":"purple","__v":0},{"_id":"68b5189329165156f06c89b6","key":"secondary","value":"orange","__v":0}]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   164  100   164    0     0  11261      0 --:--:-- --:--:-- --:--:-- 11714


In [37]:
!curl localhost:30080/api?colorkey=primary

COLOR : purple, HOSTNAME : color-api-7bf587b48f-x667r


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    53  100    53    0     0   8333      0 --:--:-- --:--:-- --:--:--  8833


In [38]:
!curl localhost:30080/api?colorkey=secondary

COLOR : orange, HOSTNAME : color-api-7bf587b48f-x667r


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    53  100    53    0     0   8450      0 --:--:-- --:--:-- --:--:--  8833


In [39]:
!kubectl scale deployment color-api --replicas=5


deployment.apps/color-api scaled


In [41]:
!kubectl get pods

NAME                         READY   STATUS    RESTARTS        AGE
color-api-7bf587b48f-2xn4r   1/1     Running   0               9s
color-api-7bf587b48f-7w5mh   1/1     Running   0               9s
color-api-7bf587b48f-9knpt   1/1     Running   0               9s
color-api-7bf587b48f-x667r   1/1     Running   2 (5m35s ago)   6m29s
color-api-7bf587b48f-zdbp6   1/1     Running   0               9s
mongodb-ss-0                 1/1     Running   0               152m


In [44]:
!curl localhost:30080/api?colorkey=secondary

COLOR : orange, HOSTNAME : color-api-7bf587b48f-9knpt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    53  100    53    0     0   3612      0 --:--:-- --:--:-- --:--:--  3785


In [45]:
!curl localhost:30080/api?colorkey=secondary

COLOR : orange, HOSTNAME : color-api-7bf587b48f-2xn4r


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    53  100    53    0     0   3249      0 --:--:-- --:--:-- --:--:--  3312


In [46]:
!kubectl apply -f .

deployment.apps/color-api unchanged
service/color-svc unchanged
statefulset.apps/mongodb-ss configured
secret/mongodb-colordb-creds unchanged
secret/mongodb-root-creds unchanged
configmap/mongodb-init-colordb unchanged
service/mongodb-svc unchanged
pod/traffic-generator created


In [47]:
!kubectl get pods

NAME                         READY   STATUS    RESTARTS      AGE
color-api-7bf587b48f-2xn4r   1/1     Running   0             4m44s
color-api-7bf587b48f-7w5mh   1/1     Running   0             4m44s
color-api-7bf587b48f-9knpt   1/1     Running   0             4m44s
color-api-7bf587b48f-x667r   1/1     Running   2 (10m ago)   11m
color-api-7bf587b48f-zdbp6   1/1     Running   0             4m44s
mongodb-ss-0                 1/1     Running   0             157m
traffic-generator            1/1     Running   0             8s


In [49]:
!kubectl logs traffic-generator

Sending request to color-svc/api every 0.5 seconds..
2025-09-01 04:02:54 COLOR : blue, HOSTNAME : color-api-7bf587b48f-2xn4r
2025-09-01 04:02:54 COLOR : blue, HOSTNAME : color-api-7bf587b48f-7w5mh
2025-09-01 04:02:55 COLOR : blue, HOSTNAME : color-api-7bf587b48f-9knpt
2025-09-01 04:02:55 COLOR : blue, HOSTNAME : color-api-7bf587b48f-zdbp6
2025-09-01 04:02:56 COLOR : blue, HOSTNAME : color-api-7bf587b48f-7w5mh
2025-09-01 04:02:56 COLOR : blue, HOSTNAME : color-api-7bf587b48f-x667r
2025-09-01 04:02:57 COLOR : blue, HOSTNAME : color-api-7bf587b48f-9knpt
2025-09-01 04:02:57 COLOR : blue, HOSTNAME : color-api-7bf587b48f-9knpt
2025-09-01 04:02:58 COLOR : blue, HOSTNAME : color-api-7bf587b48f-9knpt
2025-09-01 04:02:58 COLOR : blue, HOSTNAME : color-api-7bf587b48f-zdbp6
2025-09-01 04:02:59 COLOR : blue, HOSTNAME : color-api-7bf587b48f-zdbp6
2025-09-01 04:02:59 COLOR : blue, HOSTNAME : color-api-7bf587b48f-x667r
2025-09-01 04:03:00 COLOR : blue, HOSTNAME : color-api-7bf587b48f-2xn4r
2025-09-01 

In [51]:
!kubectl delete pod --force traffic-generator

pod "traffic-generator" force deleted


In [52]:
!kubectl apply -f .

deployment.apps/color-api unchanged
service/color-svc unchanged
statefulset.apps/mongodb-ss configured
secret/mongodb-colordb-creds unchanged
secret/mongodb-root-creds unchanged
configmap/mongodb-init-colordb unchanged
service/mongodb-svc unchanged
pod/traffic-generator created


In [53]:
!kubectl logs pod/traffic-generator

Sending request to color-svc/api?colorkey=primary every 0.5 seconds..
2025-09-01 04:04:43 COLOR : purple, HOSTNAME : color-api-7bf587b48f-7w5mh
2025-09-01 04:04:43 COLOR : purple, HOSTNAME : color-api-7bf587b48f-x667r
2025-09-01 04:04:44 COLOR : purple, HOSTNAME : color-api-7bf587b48f-x667r
2025-09-01 04:04:44 COLOR : purple, HOSTNAME : color-api-7bf587b48f-7w5mh
2025-09-01 04:04:45 COLOR : purple, HOSTNAME : color-api-7bf587b48f-9knpt
2025-09-01 04:04:45 COLOR : purple, HOSTNAME : color-api-7bf587b48f-9knpt
2025-09-01 04:04:46 COLOR : purple, HOSTNAME : color-api-7bf587b48f-9knpt
2025-09-01 04:04:46 COLOR : purple, HOSTNAME : color-api-7bf587b48f-x667r
2025-09-01 04:04:47 COLOR : purple, HOSTNAME : color-api-7bf587b48f-zdbp6
2025-09-01 04:04:47 COLOR : purple, HOSTNAME : color-api-7bf587b48f-x667r
2025-09-01 04:04:48 COLOR : purple, HOSTNAME : color-api-7bf587b48f-2xn4r
2025-09-01 04:04:48 COLOR : purple, HOSTNAME : color-api-7bf587b48f-9knpt
2025-09-01 04:04:49 COLOR : purple, HOSTNA

In [54]:
!curl -X POST http://localhost:30080/api/color/primary -H "Content-Type: application/json" -d "{\"value\":\"red\"}"

Created


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    22  100     7  100    15    463    994 --:--:-- --:--:-- --:--:--  1466


In [55]:
!kubectl logs pod/traffic-generator

Sending request to color-svc/api?colorkey=primary every 0.5 seconds..
2025-09-01 04:04:43 COLOR : purple, HOSTNAME : color-api-7bf587b48f-7w5mh
2025-09-01 04:04:43 COLOR : purple, HOSTNAME : color-api-7bf587b48f-x667r
2025-09-01 04:04:44 COLOR : purple, HOSTNAME : color-api-7bf587b48f-x667r
2025-09-01 04:04:44 COLOR : purple, HOSTNAME : color-api-7bf587b48f-7w5mh
2025-09-01 04:04:45 COLOR : purple, HOSTNAME : color-api-7bf587b48f-9knpt
2025-09-01 04:04:45 COLOR : purple, HOSTNAME : color-api-7bf587b48f-9knpt
2025-09-01 04:04:46 COLOR : purple, HOSTNAME : color-api-7bf587b48f-9knpt
2025-09-01 04:04:46 COLOR : purple, HOSTNAME : color-api-7bf587b48f-x667r
2025-09-01 04:04:47 COLOR : purple, HOSTNAME : color-api-7bf587b48f-zdbp6
2025-09-01 04:04:47 COLOR : purple, HOSTNAME : color-api-7bf587b48f-x667r
2025-09-01 04:04:48 COLOR : purple, HOSTNAME : color-api-7bf587b48f-2xn4r
2025-09-01 04:04:48 COLOR : purple, HOSTNAME : color-api-7bf587b48f-9knpt
2025-09-01 04:04:49 COLOR : purple, HOSTNA

In [56]:
!kubectl delete pod --force traffic-generator

pod "traffic-generator" force deleted


In [57]:
!kubectl scale statefulset mongodb-ss --replicas=5

statefulset.apps/mongodb-ss scaled


In [59]:
!kubectl get pods

NAME                         READY   STATUS    RESTARTS       AGE
color-api-7bf587b48f-2xn4r   1/1     Running   1 (103s ago)   9m38s
color-api-7bf587b48f-7w5mh   1/1     Running   0              9m38s
color-api-7bf587b48f-9knpt   1/1     Running   0              9m38s
color-api-7bf587b48f-x667r   1/1     Running   2 (15m ago)    15m
color-api-7bf587b48f-zdbp6   1/1     Running   0              9m38s
mongodb-ss-0                 1/1     Running   0              162m
mongodb-ss-1                 1/1     Running   0              11s
mongodb-ss-2                 1/1     Running   0              9s
mongodb-ss-3                 1/1     Running   0              7s
mongodb-ss-4                 1/1     Running   0              4s


In [60]:
!kubectl delete -f .

deployment.apps "color-api" deleted
service "color-svc" deleted
statefulset.apps "mongodb-ss" deleted
secret "mongodb-colordb-creds" deleted
secret "mongodb-root-creds" deleted
configmap "mongodb-init-colordb" deleted
service "mongodb-svc" deleted


Error from server (NotFound): error when deleting "traffic-genrator-pod.yaml": pods "traffic-generator" not found


In [61]:
!kubectl get pv,pvc

NAME                                                        CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                               STORAGECLASS   VOLUMEATTRIBUTESCLASS   REASON   AGE
persistentvolume/pvc-063ffbc4-e98c-4608-9897-d810aa369c68   5Gi        RWO            Delete           Bound    default/mongodb-data-mongodb-ss-0   hostpath       <unset>                          164m
persistentvolume/pvc-13ec6fb7-d372-4754-b734-b730e30d4c61   5Gi        RWO            Delete           Bound    default/mongodb-data-mongodb-ss-1   hostpath       <unset>                          2m29s
persistentvolume/pvc-32d436ad-a648-40f4-9a31-cffc3ba1c084   5Gi        RWO            Delete           Bound    default/mongodb-data-mongodb-ss-4   hostpath       <unset>                          2m22s
persistentvolume/pvc-44ff06f1-533d-4a70-b72c-dc585bc43214   5Gi        RWO            Delete           Bound    default/mongodb-data-mongodb-ss-2   hostpath       <unset>                         

In [62]:
!kubectl delete pvc --all

persistentvolumeclaim "mongodb-data-mongodb-ss-0" deleted
persistentvolumeclaim "mongodb-data-mongodb-ss-1" deleted
persistentvolumeclaim "mongodb-data-mongodb-ss-2" deleted
persistentvolumeclaim "mongodb-data-mongodb-ss-3" deleted
persistentvolumeclaim "mongodb-data-mongodb-ss-4" deleted


In [63]:
!kubectl get svc

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   3d3h


In [64]:
!kubectl get pods

No resources found in default namespace.


In [65]:
!kubectl get deployments

No resources found in default namespace.


In [66]:
!kubectl get configmaps

NAME               DATA   AGE
kube-root-ca.crt   1      3d3h


In [67]:
!kubectl get secrets

No resources found in default namespace.
